In this notebook, I present a simple rule for calculating derivatives for the backward propagation of all kinds of neural networks. I first derive the rule, then apply it to derive the backward propagation for different neural networks, such as plain neural networks, LSTM and CNN. Although the math of the rule is no different from the derivations you saw in anywhere else (I'm not inventing new maths of course), the main point I'd like to share is a simple way to get the backward propagation equations given a forward propagation equation. 

I know in practice most of the time you'll only need to provide the forward propagation and the frameworks will do the backward propagation for you, but I'm still interested in getting through all pieces of the neural networks, so I share some great tricks that allow you to derive the backward propagations quickly.  

## The Rule For All Neural Networks

Although the backward propagation only requires basic calculus, it still got some work there, especially for calculus with matrices. And after working the math out, we tend to forget how we get there. On the other day, we'll need to do the math again to get the results (if we had to do it by our own). However, in neural networks, actually only a few types of matrix calculus are involved, that's because we almost always use a linear layer and a non-linear layer. What makes it even simpler is, we almost always want to change the dimension in the linear layer (the layer's number of input neurons and output neurons are different) and keep the dimension in the non-linear layer (the layer's number of input neurons and output neurons are the same). That's interesting! It means we only need to find out the backward propagation for the following two types of forward propagation: 
$$
\begin{align}
Z &= WX + b \\
Z &= f(X)
\end{align}
$$

Where $X$ is the input, $Z$ is the output, $W$ is the weight matrix and $b$ is the bias, and $f$ is a non-linear function acting on $X$ element-wisely. Now we look at the two cases in detail. 

### 1. When the forward propagation is: $ Z = WX + b $

While the dimension of $X$ can be anything (usually at least 2-D), for simplicity, we first consider a simple case where $X$ is two dimensional. Let's say $X$'s shape is $(N, M)$, $Z$'s shape is $(N^\prime, M)$, $W$'s shape is $(N^\prime, N)$, $b$'s shape is $(1, M)$. We have forward propagation of this form when the input layer has $N$ neurons and the output layer has $N^\prime$ neurons, and we have $M$ samples in total.  

For the backward propagation, we need to figure out the derivatives for $X$ , $W$, and $b$, in terms of the derivatives for $Z$. The derivatives for $X$ will be used for propagating further back; the derivatives for $W$ and $b$ will be used for updating these two parameters during training. 

By "derivatives for $Z$", I mean we want to get the derivatives for every element of $Z$: ${\partial J/\partial Z_{ij}}$, where $J$ is the objective to optimize. We can use derivatives for matrix to simplify our notation. Define: 

$$\big({\partial J\over\partial Z}\big)_{ij} := {\partial J\over\partial Z_{ij}}$$

Then we can use $\partial J/\partial Z$, a matrix, to represent all the derivative terms.  

#### 1.1 Derivatives for $X$

Our goal here is to derive ${\partial J/\partial X}$ given ${\partial J/\partial Z}$. By the chain rule, we have: 

$${\partial J\over\partial X} = {\partial J\over\partial Z}{\partial Z\over\partial X}$$

First, we calculate ${\partial Z/\partial X}$. This derivative is matrix respect to matrix, we may have a different mathematical definition for general cases, however, in here, I'd like to define it as: 

$$\big({\partial Z\over\partial X}\big)_{ijm} = {\partial Z_{im}\over\partial X_{jm}}$$

Why? Because the last dimension of $X$ and $Z$ are the input samples, when we train the model, we do it sample by sample. We're not interested in derivatives across different samples. 

With this definition, now we can apply normal calculus:

$$
\begin{aligned}
\big({\partial Z\over\partial X}\big)_{ijm} &= {\partial Z_{im}\over\partial X_{jm}} \\
&= {\partial \big(\sum_k W_{ik} X_{km} + b_i\big)\over\partial X_{jm}} \\
&= W_{ij}
\end{aligned}
$$

Then use the chain rule:

$$
\begin{aligned}
\big({\partial J\over\partial X}\big)_{im} &= \big({\partial J\over\partial Z} {\partial Z\over\partial X}\big)_{ijm} \\
&= \sum_j \big({\partial J\over\partial Z}\big)_{jm}\big({\partial Z\over\partial X}\big)_{jim} \\
&= \sum_j W_{ji} \big({\partial J\over\partial Z}\big)_{jm}
\end{aligned}
$$

Note if $A_{im} = \sum_j B_{ji} C_{jm}$, then $A=B^T C$. So we have:

$$
{\partial J\over\partial X} = W^T {\partial J\over\partial Z}
$$


#### 1.2 Derivatives for W

Here we want to derive ${\partial J/\partial W}$ given ${\partial J/\partial Z}$. By the chain rule, we have: 

$${\partial J\over\partial W} = {\partial J\over\partial Z}{\partial Z\over\partial W}$$

First, we calculate ${\partial Z/\partial W}$. 

$$
\begin{aligned}
\big({\partial Z\over\partial W}\big)_{kmij} &= {\partial Z_{km}\over\partial W_{ij}} \\
&= {\partial \big(\sum_n W_{kn} X_{nm} + b_k\big)\over\partial W_{ij}} \\
&= \delta_{ki}X_{jm}
\end{aligned}
$$

Where $\delta_{ij} = 1$ if $i = j$, otherwise $\delta_{ij} = 0$. Now use the chain rule:

$$
\begin{aligned}
\big({\partial J\over\partial W}\big)_{ij} &= \big({\partial J\over\partial Z} {\partial Z\over\partial W}\big)_{ij} \\
&= \sum_{km} \big({\partial J\over\partial Z}\big)_{km}\big({\partial Z\over\partial W}\big)_{kmij} \\
&= \sum_{km} \big({\partial J\over\partial Z}\big)_{km}\delta_{ki}X_{jm} \\
&= \sum_m \big({\partial J\over\partial Z}\big)_{im} X_{jm}
\end{aligned}
$$

The sum of the last dimension means a sum over all samples. Note if $A_{ij} = \sum_j B_{im} C_{jm}$, then $A=B C^T$. So: 

$$
{\partial J\over\partial W} = {\partial J\over\partial Z} X^T
$$


#### 1.3 Derivatives for b

In here we'd like to derive ${\partial J/\partial b}$ given ${\partial J/\partial Z}$. By the chain rule, we have: 

$${\partial J\over\partial b} = {\partial J\over\partial Z}{\partial Z\over\partial b}$$

First, we calculate ${\partial Z/\partial b}$.

$$
\begin{aligned}
\big({\partial Z\over\partial b}\big)_{ijm} &= {\partial Z_{im}\over\partial b_{j}} \\
&= {\partial \big(\sum_k W_{ik} X_{km} + b_i\big)\over\partial b_{j}} \\
&= \delta_{ij}
\end{aligned}
$$

Apply the chain rule:

$$
\begin{aligned}
\big({\partial J\over\partial b}\big)_{i} &= \big({\partial J\over\partial Z} {\partial Z\over\partial b}\big)_{i} \\
&= \sum_{jm} \big({\partial J\over\partial Z}\big)_{jm}\big({\partial Z\over\partial b}\big)_{jim} \\
&= \sum_{jm} \delta_{ij}\big({\partial J\over\partial Z}\big)_{jm} \\
&= \sum_m \big({\partial J\over\partial Z}\big)_{im}
\end{aligned}
$$

Again, the sum of the last dimension means a sum over all samples. 


#### 1.4 Summary

In summary, if the forward propagation is in the form of:
$$ Z = W X + b$$

Then:
$$
\begin{aligned}
{\partial J\over\partial X} &= W^T {\partial J\over\partial Z} \\
{\partial J\over\partial W} &= {\partial J\over\partial Z} X^T \\
\big({\partial J\over\partial b}\big)_{i} &= \sum_m \big({\partial J\over\partial Z}\big)_{im}
\end{aligned}
$$

### 2. When the forward propagation is: $ A = f(Z) $

The forward propagation is this form when we apply activations. Usually $f$ is a non-linear funtion, and it's applied to every element of $Z$. So for the derivatives, we can calculate element-wisely. The derivatives calculation is then pretty straightforward: 

$${\partial J\over\partial Z} = {\partial J\over\partial A} * f^\prime(Z) $$

"$*$" means element-wise multiplication. $f^\prime(Z)$ has the same dimension as $A$ and $Z$, so it's valid to perform element-wise multiplication. 

We look at some most commonly used non-linear functions. 

#### 2.1 Sigmoid

The derivative for the sigmoid function is: 

$$\sigma^\prime(X) = \sigma(X) * (1 - \sigma(X))$$

Then we have: 

$${\partial J\over\partial Z} = {\partial J\over\partial A} * A * (1 - A) $$

#### 2.2 Tanh

The derivative for the $\tanh$ function is: 

$$\tanh^\prime(X) = 1 - \tanh(X)^2$$

Then we have: 

$${\partial J\over\partial Z} = {\partial J\over\partial A} * (1 - A^2) $$


### 3. Summary

Another notation is often used for the partial derivatives against the objective function:
$$dX := {\partial J\over\partial X}$$

I don't quite like this notation because $dX$ has different meanings in math and it's sometimes confusing. But since this notation is widely used already and is more concise, I'll summarize our rules using this notation:



<div style="border: 2px solid #A4A4A4; padding: 5px">
**Rule 1**: <br />
Forward propagation: $$Z = WX + b$$
$$\Downarrow$$
Backward propagation:
$$
\begin{aligned}
dX &= W^T dZ \\
dW &= dZ \, X^T \\
\big(db\big)_{i} &= \sum_m \big(dZ\big)_{im}
\end{aligned}
$$

**Rule 2**: <br />
Forward propagation: $$A = f(Z)$$
$$\Downarrow$$
Backward propagation:
$$dZ = dA * f^\prime(Z) $$
</div>

For rule 1, just put $dZ$ on the right hand side and multiply a transposed matrix, to match the dimension. Rule 2 is pretty straightforward, there's no change in dimension. Isn't it simple? 

Great, now let's use this to derive the backward propagation for different types of neural networks. You'll see the power of the simple rules when we come to complicated forward propagations. 

## Application:  A Simple Example

Now we apply the rules to derive the backward propagation for a simple example: a neural networks with a stack of fully connected (FC) layers. 

The forward propagation is:
$$
\begin{align}
a^{[1]} &= relu(W_{1} X + b_1)\tag{s.1} \\
a^{[2]} &= relu(W_{2} a^{[1]} + b_2)\tag{s.2} \\
y &= sigmoid(a^{[2]})\tag{s.3}
\end{align}
$$

$a^{[i]}$ means the $a$ of the $i$'th layer. 

For Eq. (s.3), by rule (2):

$${\partial J\over\partial a^{[2]}} = {\partial J\over\partial y} * y * (1-y)$$

The value of ${\partial J/\partial y}$ depends on the definition of the loss function $J(y, y^*)$, where $y$ is the model output and $y^*$ is the label of the training sample. 

For Eq. (s.2), let $Z^{[2]} = W_{2} a^{[1]} + b_2$, then $a^{[2]} = relu(Z^{[2]})$. By rule (1):
$${\partial J\over\partial Z^{[2]}} = {\partial J\over\partial a^{[2]}} relu^\prime(Z^{[2]})$$

Then by rule (2):
$$
\begin{align}
{\partial J\over\partial a^{[1]}} &= {\partial J\over\partial a^{[2]}} relu^\prime(Z^{[2]}) W_{2}^T a^{[2]} \\
{\partial J\over\partial W_{2}} &= {\partial J\over\partial a^{[2]}} relu^\prime(Z^{[2]}) a^{[2]} (a^{[1]})^T \\
{\partial J\over\partial b_{2}} &= {\partial J\over\partial a^{[2]}} relu^\prime(Z^{[2]}) \sum_m a^{[2]} 
\end{align}
$$

Same for Eq. (s.1):
$$
\begin{align}
{\partial J\over\partial X} &= {\partial J\over\partial a^{[1]}} relu^\prime(Z^{[1]}) W_{1}^T a^{[1]} \\
{\partial J\over\partial W_{1}} &= {\partial J\over\partial a^{[1]}} relu^\prime(Z^{[1]}) X (a^{[1]})^T \\
{\partial J\over\partial b_{2}} &= {\partial J\over\partial a^{[1]}} relu^\prime(Z^{[1]}) \sum_m a^{[1]} 
\end{align}
$$

Where $Z^{[1]} = W_{1} X + b_1$. You can easily generalize these equations to deeper neural networks. You can just replace $relu$ (and its derivative) if a different non-linear function is used for activation. 

## Application: RNN

### 1 Simple RNN

A simple plain RNN unit takes $x^{\langle t \rangle} $ and $a^{\langle t-1 \rangle}$ as input at time $t$, and returns $a^{\langle t \rangle}$ as output. $a^{\langle t \rangle}$ is RNN cell's hidden state at time $t$. If the RNN has $n$ units, and we feed $m$ samples, the shape of $a^{\langle t \rangle}$ is $(m, n)$. The trainable parameters are $W_{aa}$, $W_{ax}$ and $b_a$. 

The forward propagation is: 
$$a^{\langle t \rangle} = \tanh(W_{aa} a^{\langle t-1 \rangle} + W_{ax} x^{\langle t \rangle} + b_a)$$

For this form matches the form of our rules, the only difference is instead of $WX + b$, we have $W_A A + W_X X + b$, but when taking derivatives of one variable, we can treat other variables as constants, so we can apply the rules directly to get the backward propagation: 

$${\partial J\over\partial W_{aa}} = \left(1 - (a^{\langle t \rangle})^2\right){\partial J \over\partial a^{\langle t \rangle}}\left(a^{\langle t - 1 \rangle}\right)^T$$
$${\partial J\over\partial a^{\langle t - 1 \rangle}} =  \left(1 - (a^{\langle t \rangle})^2\right) W_{aa}^T {\partial J \over\partial a^{\langle t \rangle}}$$
$${\partial J\over\partial W_{ax}} = \left(1 - (a^{\langle t \rangle})^2\right){\partial J \over\partial a^{\langle t \rangle}}\left(x^{\langle t \rangle}\right)^T$$
$${\partial J\over\partial x^{\langle t \rangle}} = \left(1 - (a^{\langle t \rangle})^2\right) W_{ax}^T{\partial J \over\partial a^{\langle t \rangle}}$$
$${\partial J\over\partial b_a} =  \sum \left(1 - (a^{\langle t \rangle})^2\right){\partial J \over\partial a^{\langle t \rangle}}$$

Note that the ${\partial J /\partial a^{\langle t-1 \rangle}}$ in here just contains the gradients from the next cell (at time $t$); we also have gradients from this cell's output $\hat{y}^{\langle t-1 \rangle}$, where: 

$$\hat{y}^{\langle t-1 \rangle} = sigmoid(W_{ya} a^{\langle t-1 \rangle} + b_y)$$

Of course the activation can also be other non-linearities (e.g., softmax). By our rules, we have:
$${\partial J\over\partial a^{\langle t - 1 \rangle}} = \hat{y}^{\langle t - 1 \rangle} * \left(1-\hat{y}^{\langle t - 1 \rangle}\right) * W_{ya}^T{\partial J\over\partial \hat{y}^{\langle t - 1 \rangle}}$$

Combining the contributions from both next cell and the output of the current cell, we have:  
$${\partial J\over\partial a^{\langle t - 1 \rangle}} = \left(1 - (a^{\langle t \rangle})^2\right) W_{aa}^T {\partial J \over\partial a^{\langle t \rangle}} + \hat{y}^{\langle t - 1 \rangle} * \left(1-\hat{y}^{\langle t - 1 \rangle}\right) *W_{ya}^T{\partial J\over\partial \hat{y}^{\langle t - 1 \rangle}}$$

As a side note, we can also write the RNN forward propagation in a compact form: 
$$a^{\langle t \rangle} = \tanh(\begin{bmatrix} W_{aa} W_{ax} \end{bmatrix} \begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle}\end{bmatrix} + b_a)$$
$$\hat{y}^{\langle t \rangle} = sigmoid(W_{ya} a^{\langle t \rangle} + b_y)$$

Then it's even more straightforward to apply our rules. 

### 2 LSTM

A LSTM cell takes $x^{\langle t \rangle}$, $a^{\langle t-1 \rangle}$ and $c^{\langle t-1 \rangle}$ as inputs, returns $a^{\langle t \rangle}$, $c^{\langle t \rangle}$ as outputs, at time $t$. In here, $a^{\langle t \rangle}$ is the hidden state, $c^{\langle t \rangle}$ is the cell state, they have the same dimension. If the LSTM has $n$ units, and we feed $m$ samples, the shape of $a^{\langle t \rangle}$ is $(m, n)$. 

The forward propagation is:

$$\Gamma_u^{\langle t \rangle} = \sigma\left(W_u\begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle}\end{bmatrix} + b_u \right)\tag{r.2.1}$$
$$\Gamma_f^{\langle t \rangle} = \sigma\left(W_f\begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle}\end{bmatrix} + b_f \right)\tag{r.2.2}$$
$$\Gamma_o^{\langle t \rangle} = \sigma\left(W_o\begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle}\end{bmatrix} + b_o \right)\tag{r.2.3}$$
$$\widetilde{c}^{\langle t \rangle} = \tanh\left(W_c\begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle}\end{bmatrix} + b_c \right)\tag{r.2.4}$$

$$c^{\langle t \rangle} = \Gamma_u^{\langle t \rangle} * \widetilde{c}^{\langle t \rangle} + \Gamma_f^{\langle t \rangle} * c^{\langle t-1 \rangle}\tag{r.2.5}$$
$$a^{\langle t \rangle} = \Gamma_o^{\langle t \rangle} * \tanh\left(c^{\langle t \rangle}\right)\tag{r.2.6}$$

Where $\Gamma_u^{\langle t \rangle}$ is the update gate, $\Gamma_f^{\langle t \rangle}$ is the forget gate.  If $\Gamma_u^{\langle t \rangle} = 0$ and $\Gamma_f^{\langle t \rangle} = 1$, then the new cell state equals to the previous cell state; If $\Gamma_u^{\langle t \rangle} = 1$ and $\Gamma_f^{\langle t \rangle} = 0$, then the previous cell state will be completely forgotten. 
$\Gamma_o^{\langle t \rangle}$ is the output gate, this gate multiplies by the cell state gives the new hidden state. 
All the weight matrices $W$ and bias vectors $b$ are trainable parameters. The gate notations are from Andrew Ng. 

#### 2.1 The update gate
We first look at $W_u$ and $b_u$. From Eq. (r.2.1), our rules can be applied directly to get: 

$${\partial J\over\partial W_{u}} = {\partial J\over\partial \Gamma_u^{\langle t \rangle}}\begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle}\end{bmatrix}^T * \Gamma_u^{\langle t \rangle} * \big(1-\Gamma_u^{\langle t \rangle}\big)\tag{r.2.7}$$

We need to use ${\partial J/\partial a^{\langle t \rangle}}$ and ${\partial J/\partial c^{\langle t \rangle}}$ to figure out ${\partial J/\partial \Gamma_u^{\langle t \rangle}}$, because the outputs are $a^{\langle t \rangle}$ and $c^{\langle t \rangle}$, and we're propagating backwards. 

From Eq. (r.2.5), we have: 
$${\partial J\over\partial \Gamma_u^{\langle t \rangle}} = {\partial J\over\partial c^{\langle t \rangle}} * \widetilde{c}^{\langle t \rangle}$$

Here we need to be careful, because not only ${\partial J/\partial c^{\langle t \rangle}}$ has contribution to ${\partial J/\partial \Gamma_u^{\langle t \rangle}}$, ${\partial J/\partial a^{\langle t \rangle}}$ also does. As we can see from Eq. (r.2.6), we have: 
$$
{\partial J\over\partial c^{\langle t \rangle}} = {\partial J\over\partial a^{\langle t \rangle}} * \Gamma_o^{\langle t \rangle} *\big(1-\tanh(c^{\langle t \rangle})^2\big)
$$

Sum it up:
$${\partial J\over\partial \Gamma_u^{\langle t \rangle}} = \big({\partial J\over\partial c^{\langle t \rangle}} + {\partial J\over\partial a^{\langle t \rangle}} * \Gamma_o^{\langle t \rangle} *\big(1-\tanh(c^{\langle t \rangle})^2\big)\big) * \widetilde{c}^{\langle t \rangle}\tag{r.2.8}$$

We can then plug it to Eq. (r.2.7) to get ${\partial J/\partial W_{u}}$.

By our rules, it's easy to get:

$${\partial J\over\partial b_{u}} = \sum_m {\partial J\over\partial \Gamma_u^{\langle t \rangle}} * \Gamma_u^{\langle t \rangle} * \big(1-\Gamma_u^{\langle t \rangle}\big)$$

Where ${\partial J/\partial \Gamma_u^{\langle t \rangle}}$ is in Eq. (r.2.8). 

#### 2.2 The forget gate
Now we look at $W_f$ and $b_f$, the process will be very similar to the update gate. From Eq. (r.2.2), our rules can be applied directly to get: 

$${\partial J\over\partial W_{f}} = {\partial J\over\partial \Gamma_f^{\langle t \rangle}}\begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle}\end{bmatrix}^T * \Gamma_f^{\langle t \rangle} * \big(1-\Gamma_f^{\langle t \rangle}\big)\tag{r.2.9}$$

From Eq. (r.2.5), we have: 
$${\partial J\over\partial \Gamma_f^{\langle t \rangle}} = {\partial J\over\partial c^{\langle t \rangle}} * c^{\langle t-1 \rangle}$$

Similarly, we also have contribution from Eq. (r.2.6): 
$$
{\partial J\over\partial c^{\langle t \rangle}} = {\partial J\over\partial a^{\langle t \rangle}} * \Gamma_o^{\langle t \rangle} *\big(1-\tanh(c^{\langle t \rangle})^2\big)
$$

So finally:
$${\partial J\over\partial \Gamma_f^{\langle t \rangle}} = \big({\partial J\over\partial c^{\langle t \rangle}} + {\partial J\over\partial a^{\langle t \rangle}} * \Gamma_o^{\langle t \rangle} *\big(1-\tanh(c^{\langle t \rangle})^2\big)\big) * c^{\langle t-1 \rangle}\tag{r.2.10}$$

We can then plug it to Eq. (r.2.9) to get ${\partial J/\partial W_{f}}$.

For ${\partial J/\partial b_{f}}$:

$${\partial J\over\partial b_{f}} = \sum_m {\partial J\over\partial \Gamma_f^{\langle t \rangle}} * \Gamma_f^{\langle t \rangle} * \big(1-\Gamma_f^{\langle t \rangle}\big)$$

Where ${\partial J/\partial \Gamma_f^{\langle t \rangle}}$ is in Eq. (r.2.10). 

#### 2.3 The output gate

This one is easy, because it's not in Eq. (r.2.5). From Eq. (r.2.3): 

$${\partial J\over\partial W_{o}} = {\partial J\over\partial \Gamma_o^{\langle t \rangle}}\begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle}\end{bmatrix}^T * \Gamma_o^{\langle t \rangle} * \big(1-\Gamma_o^{\langle t \rangle}\big)$$

From Eq. (r.2.6): 
$$
{\partial J\over\partial \Gamma_o^{\langle t \rangle}} = {\partial J\over\partial a^{\langle t \rangle}} * \tanh(c^{\langle t \rangle})\tag{r.2.11}
$$

So:
$${\partial J\over\partial W_{o}} = {\partial J\over\partial a^{\langle t \rangle}} * \tanh(c^{\langle t \rangle}) * \Gamma_o^{\langle t \rangle} * \big(1-\Gamma_o^{\langle t \rangle}\big) \begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle}\end{bmatrix}^T$$

Then:
$${\partial J\over\partial b_{o}} = \sum_m{\partial J\over\partial a^{\langle t \rangle}} * \tanh(c^{\langle t \rangle}) * \Gamma_o^{\langle t \rangle} * \big(1-\Gamma_o^{\langle t \rangle}\big) $$

#### 2.4 The candidate cell state

We're almost there! From Eq. (r.2.4): 

$${\partial J\over\partial W_c} = {\partial J\over\partial \widetilde{c}^{\langle t \rangle}}\begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle}\end{bmatrix}^T * \left(1- \left(\widetilde{c}^{\langle t \rangle}\right)^2\right)$$

From Eq. (r.2.5), we have: 
$${\partial J\over\partial \widetilde{c}^{\langle t \rangle}} = {\partial J\over\partial c^{\langle t \rangle}} * \Gamma_u^{\langle t \rangle}$$

Similarly, we also have contribution from Eq. (r.2.6): 
$$
{\partial J\over\partial c^{\langle t \rangle}} = {\partial J\over\partial a^{\langle t \rangle}} * \Gamma_o^{\langle t \rangle} *\big(1-\tanh(c^{\langle t \rangle})^2\big)
$$

Sum it up:
$${\partial J\over\partial \widetilde{c}^{\langle t \rangle}} = \left({\partial J\over\partial c^{\langle t \rangle}} + {\partial J\over\partial a^{\langle t \rangle}} * \Gamma_o^{\langle t \rangle} *\left(1-\tanh(c^{\langle t \rangle})^2\right)\right) * \Gamma_u^{\langle t \rangle}\tag{r.2.12}$$

For ${\partial J/\partial b_{c}}$:

$${\partial J\over\partial b_{c}} = \sum_m {\partial J\over\partial \widetilde{c}^{\langle t \rangle}}* \left(1- \left(\widetilde{c}^{\langle t \rangle}\right)^2\right)$$

Where ${\partial J/\partial \widetilde{c}^{\langle t \rangle}}$ is in Eq. (r.2.12). 

#### 2.5 Input, hidden state and cell state

This is the final step! First we look at the derivatives for $a^{\langle t-1 \rangle}$ and $x^{\langle t \rangle}$. They always appear together in Eq. (r.2.1-4), so we have: 

$$
\begin{align}
{\partial J\over\partial\begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle}\end{bmatrix}} = & W_u^T {\partial J\over\partial \Gamma_u^{\langle t \rangle}} * \Gamma_u^{\langle t \rangle} * \big(1-\Gamma_u^{\langle t \rangle}\big) \\
& + W_f^T {\partial J\over\partial \Gamma_f^{\langle t \rangle}} * \Gamma_f^{\langle t \rangle} * \big(1-\Gamma_f^{\langle t \rangle}\big) \\
& + W_o^T {\partial J\over\partial \Gamma_o^{\langle t \rangle}} * \Gamma_o^{\langle t \rangle} * \big(1-\Gamma_o^{\langle t \rangle}\big) \\
& + W_c^T{\partial J\over\partial \widetilde{c}^{\langle t \rangle}}* \left(1- \left(\widetilde{c}^{\langle t \rangle}\right)^2\right)
\end{align}
$$

The partial derivatives on the right hand side are already calculated in Eq. (2.1.8) (2.1.10) (2.1.11) (2.1.12).

Now we calculate the derivative for $c^{\langle t-1 \rangle}$. From Eq. (r.2.5), we have:
$$
{\partial J\over\partial c^{\langle t-1 \rangle}} = {\partial J\over\partial c^{\langle t \rangle}} * \Gamma_f^{\langle t \rangle}
$$

Similar to previous sections, $a^{\langle t \rangle}$ contributes to ${\partial J/\partial c^{\langle t \rangle}}$ as well, by Eq. (r.2.6). And:

$$
{\partial J\over\partial c^{\langle t \rangle}} = {\partial J\over\partial a^{\langle t \rangle}} * \Gamma_o^{\langle t \rangle} *\big(1-\tanh(c^{\langle t \rangle})^2\big)
$$

Sum the two terms up:
$$
{\partial J\over\partial c^{\langle t-1 \rangle}} = \left({\partial J\over\partial c^{\langle t \rangle}} + {\partial J\over\partial a^{\langle t \rangle}} * \Gamma_o^{\langle t \rangle} *\left(1-\tanh(c^{\langle t \rangle})^2\right) \right)* \Gamma_f^{\langle t \rangle}
$$

And that's it!!

#### 2.6 LSTM backward propagation summary

To sum up, the derivatives for the gates are:
$$
\begin{align}
{\partial J\over\partial \Gamma_u^{\langle t \rangle}} &= \left({\partial J\over\partial c^{\langle t \rangle}} + {\partial J\over\partial a^{\langle t \rangle}} * \Gamma_o^{\langle t \rangle} *\left(1-\tanh(c^{\langle t \rangle})^2\right)\right) * \widetilde{c}^{\langle t \rangle} \\ 
{\partial J\over\partial \Gamma_f^{\langle t \rangle}} &= \left({\partial J\over\partial c^{\langle t \rangle}} + {\partial J\over\partial a^{\langle t \rangle}} * \Gamma_o^{\langle t \rangle} *\left(1-\tanh(c^{\langle t \rangle})^2\right)\right) * c^{\langle t-1 \rangle} \\
{\partial J\over\partial \Gamma_o^{\langle t \rangle}} &= {\partial J\over\partial a^{\langle t \rangle}} * \tanh(c^{\langle t \rangle}) \\
{\partial J\over\partial \widetilde{c}^{\langle t \rangle}} &= \left({\partial J\over\partial c^{\langle t \rangle}} + {\partial J\over\partial a^{\langle t \rangle}} * \Gamma_o^{\langle t \rangle} *\left(1-\tanh(c^{\langle t \rangle})^2\right)\right) * \Gamma_u^{\langle t \rangle}
\end{align}
$$


Derivatives for the $W$'s: 
$${\partial J\over\partial W_{u}} = {\partial J\over\partial \Gamma_u^{\langle t \rangle}}\begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle}\end{bmatrix}^T * \Gamma_u^{\langle t \rangle} * \big(1-\Gamma_u^{\langle t \rangle}\big)$$
$${\partial J\over\partial W_{f}} = {\partial J\over\partial \Gamma_f^{\langle t \rangle}}\begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle}\end{bmatrix}^T * \Gamma_f^{\langle t \rangle} * \big(1-\Gamma_f^{\langle t \rangle}\big)$$
$${\partial J\over\partial W_{o}} = {\partial J\over\partial \Gamma_o^{\langle t \rangle}}\begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle}\end{bmatrix}^T * \Gamma_o^{\langle t \rangle} * \big(1-\Gamma_o^{\langle t \rangle}\big)$$
$${\partial J\over\partial W_c} = {\partial J\over\partial \widetilde{c}^{\langle t \rangle}}\begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle}\end{bmatrix}^T * \left(1- \left(\widetilde{c}^{\langle t \rangle}\right)^2\right)$$

Derivatives for the $b$'s: 
$$
\begin{align}
{\partial J\over\partial b_{u}} &= \sum_m {\partial J\over\partial \Gamma_u^{\langle t \rangle}} * \Gamma_u^{\langle t \rangle} * \big(1-\Gamma_u^{\langle t \rangle}\big) \\
{\partial J\over\partial b_{f}} &= \sum_m {\partial J\over\partial \Gamma_f^{\langle t \rangle}} * \Gamma_f^{\langle t \rangle} * \big(1-\Gamma_f^{\langle t \rangle}\big) \\
{\partial J\over\partial b_{o}} &= \sum_m{\partial J\over\partial \Gamma_o^{\langle t \rangle}} * \Gamma_o^{\langle t \rangle} * \big(1-\Gamma_o^{\langle t \rangle}\big) \\
{\partial J\over\partial b_{c}} &= \sum_m {\partial J\over\partial \widetilde{c}^{\langle t \rangle}}* \left(1- \left(\widetilde{c}^{\langle t \rangle}\right)^2\right)
\end{align}
$$

Derivatives for the input and hidden state: 
$$
\begin{align}
{\partial J\over\partial\begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle}\end{bmatrix}} = & W_u^T {\partial J\over\partial \Gamma_u^{\langle t \rangle}} * \Gamma_u^{\langle t \rangle} * \big(1-\Gamma_u^{\langle t \rangle}\big) \\
& + W_f^T {\partial J\over\partial \Gamma_f^{\langle t \rangle}} * \Gamma_f^{\langle t \rangle} * \big(1-\Gamma_f^{\langle t \rangle}\big) \\
& + W_o^T {\partial J\over\partial \Gamma_o^{\langle t \rangle}} * \Gamma_o^{\langle t \rangle} * \big(1-\Gamma_o^{\langle t \rangle}\big) \\
& + W_c^T{\partial J\over\partial \widetilde{c}^{\langle t \rangle}}* \left(1- \left(\widetilde{c}^{\langle t \rangle}\right)^2\right)
\end{align}
$$

Derivatives for the cell state: 
$$
{\partial J\over\partial c^{\langle t-1 \rangle}} = \left({\partial J\over\partial c^{\langle t \rangle}} + {\partial J\over\partial a^{\langle t \rangle}} * \Gamma_o^{\langle t \rangle} *\left(1-\tanh(c^{\langle t \rangle})^2\right) \right)* \Gamma_f^{\langle t \rangle}
$$


## Application:  CNN

### 1 Convolution

In each step of the convolution operation, we obtain one element of the next layer by calculating the sum of the direct multiplication of a part of the current layer and the filter. Let's say this part of the current layer is $A_{sub}^{[n]}$, the filter is $F$, then the forward propagation is:
$$A^{[n + 1]}_{ij} = \sum A_{sub}^{[n]} * F$$

If we only look at this equation, the backward propagation is very straightforward: 
$$
{\partial J\over\partial A_{sub}^{[n]}} = {\partial J\over\partial A^{[n + 1]}_{ij}} * F
$$

Note $A^{[n + 1]}_{ij}$ is a scalar, $A_{sub}^{[n]}$ and $F$ are matrices of the same shape. So applying this equation to the $(i, j)$'th element of $A^{[n + 1]}$ gives us the gradients of a part of $A^{[n]}$. And all convolutoin steps are independent, so we just need to loop through all $A_{sub}^{[n]}$'s elements and sum the results up! 

I can also write it in this form:

$$
{\partial J\over\partial A^{[n]}} = \sum_{i, j} {\partial J\over\partial A^{[n + 1]}_{ij}} * M(i, j)
$$

Where $M(i, j)$ has the same shape of $A^{[n]}$. For the part of $A^{[n]}$ used to the convolution step for calculating $(i, j)$, $M(i, j)_{sub} = F$; outside of this part, values of $M(i, j)$ are all zeros. This means just to propagate the gradients back to where it comes from.  


### 2 Max pooling

The max pooling is very similar to the convolution, but instead of multiplying the filters and sum it, we just take the maximum element in the original layer as the element for the next layer. So the forward propagation is:
$$A^{[n + 1]}_{ij} = \max( A_{sub}^{[n]} )$$

So the backward propagation is: 
$$
{\partial J\over\partial A_{i^\prime j^\prime}^{[n]}} = {\partial J\over\partial A^{[n + 1]}_{ij}}
$$

Where $(i^\prime, j^\prime)$ is that maximum element of $A_{sub}^{[n]}$. In other words, you just need to assign the gradient from next layer to where it comes from in the previous layer, and accumulate. 


## More Applications

We can also to the same thing for the dropout layers, batch normalization, and Residue Neural Networks. I have derived truly marvelous results for all of them, which this notebook's margin is too narrow to contain. 

## Afterword

If you reach here and are still awake and reading, you're probably crazy, but of course I'm with you :). All the derivations here are not going to be very useful practically: even if you're implementing a new neural network architecture, unless you do everything by yourself, you'll probably only need to do the forward propagation, and never need to worry about the backward propagation. However the math is not that hard at all, if you go through it, the neural networks won't be a mysterious blackbox anymore. I hope it helps, thanks for reading! 
